### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [1]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

Current working directory: d:\projects\RUNI\Thesis
Datasets path: d:\projects\RUNI\Thesis\datasets
Output path: d:\projects\RUNI\Thesis\output


In [2]:
from python.hpc import HybridArray

Numba version: 0.60.0
numba.njit is available.
CUDA is available and will be used for GPU operations.
Printing CUDA active device attributes:
    Name:                               NVIDIA GeForce GTX 1650
    Free Memory:                        3367680 [KB]
    Total Memory:                       4193984 [KB]
    Compute capability:                 7.5
    Clock rate:                         1560.00 [MHz]
    Memory clock rate:                  4001.00 [MHz]
    Memory bus width:                   128 bits
    Memory band width (theoretical)     128.03 [GByte/Sec]
    Number of multiprocessors:          16
    Minimal grid size:                  128
    Maximum grid size:                  (2147483647, 65535, 65535)
    Maximum block dimensions:           (1024, 1024, 64)
    Maximum threads per block:          1024
    Warp size:                          32
    Maximum shared memory per block:    49152 [bytes]
    Maximum registers per block:        65536
    Total constant memory:   

In [3]:
import numpy as np
from scipy.stats import chisquare, combine_pvalues

def combined_randomness_test(samples):
    """
    Computes a combined p-value for the randomness of unsigned 64-bit integers.
    
    Parameters:
        samples (np.array): Array of uint64 values from your RNG.
        
    Returns:
        float: A combined p-value using Fisher's method.
               A high p-value (> 0.05) suggests randomness.
    """
    # Convert samples to bits and reshape: one row per 64-bit number
    bits = np.unpackbits(samples.view(np.uint8)).reshape(-1, 64)
    n_samples = samples.shape[0]
    p_values = []
    
    # Expected count for ones and zeros is n_samples/2 each.
    expected = [n_samples / 2, n_samples / 2]
    
    # Test each of the 64 bit positions
    for bit in range(64):
        ones = np.sum(bits[:, bit])
        zeros = n_samples - ones
        _, p = chisquare([ones, zeros], f_exp=expected)
        p_values.append(p)
    
    # Combine the 64 p-values using Fisher's method
    _, combined_p = combine_pvalues(p_values, method='fisher')
    return combined_p

# Testing random integers in series

In [4]:
from python.random_integers.random_integers import random_integers_series
num_monte = 10
N = 100000
data_gpu = HybridArray().realloc(shape=(N,), dtype=np.uint64, use_gpu=True)
random_integers_series(data=data_gpu, seed=0)


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [5]:
combined_p = combined_randomness_test(data_gpu.numpy())
print(f'{combined_p=}')


combined_p=0.36841400743026476


# Test random matrix

In [6]:
from python.random_integers.random_integers import random_integers_matrix, random_integers_2_p_values
from python.array_math_utils.array_math_utils import average_column
num_monte = 100
N = 100000
max_num_steps = 5
data_gpu = HybridArray().realloc(shape=(num_monte,N), dtype=np.uint64, use_gpu=True)
p_values = HybridArray()
average_col = HybridArray()
for num_steps in range(1,max_num_steps+1):
    print(f'{num_steps=}')
    random_integers_matrix(data=data_gpu.uncrop(), offset_row0=0, offset_col0=0, num_steps=num_steps)
    random_integers_2_p_values(integers=data_gpu, p_values=p_values)
    if data_gpu.shape()[0] <= 10:
        average_column(p_values, out_column=average_col)
        print(f'{average_col.numpy()=}')
    p_value_per_row = []
    for ind_row in range(num_monte):
        data_row = data_gpu.select_row(ind_row).numpy()
        combined_p = combined_randomness_test(data_row.reshape(-1))
        if num_monte <= 100:
            print(f'{ind_row=} {data_row.shape=} {combined_p=}')
        p_value_per_row.append(combined_p)
    _, final_p_value = combine_pvalues(p_value_per_row, method='fisher')
    print(f'Final combined p_value: {final_p_value}')

        


num_steps=1
ind_row=0 data_row.shape=(1, 100000) combined_p=0.11454787291214055
ind_row=1 data_row.shape=(1, 100000) combined_p=0.3723270107473706
ind_row=2 data_row.shape=(1, 100000) combined_p=0.5952699358087439
ind_row=3 data_row.shape=(1, 100000) combined_p=0.26472908025839276
ind_row=4 data_row.shape=(1, 100000) combined_p=0.7476549021619403
ind_row=5 data_row.shape=(1, 100000) combined_p=0.9450179029316057
ind_row=6 data_row.shape=(1, 100000) combined_p=0.4372928760004406
ind_row=7 data_row.shape=(1, 100000) combined_p=0.19571441505236323
ind_row=8 data_row.shape=(1, 100000) combined_p=0.6966138023018993
ind_row=9 data_row.shape=(1, 100000) combined_p=0.9189767436137848
ind_row=10 data_row.shape=(1, 100000) combined_p=0.6846888197294341
ind_row=11 data_row.shape=(1, 100000) combined_p=0.9565833028015446
ind_row=12 data_row.shape=(1, 100000) combined_p=0.829350619665124
ind_row=13 data_row.shape=(1, 100000) combined_p=0.35401298094611455
ind_row=14 data_row.shape=(1, 100000) combi